# Normalziing with NLU 

The Normalizer cleans text data from dirty characters, lowercases it by default and removes punctuation.       

### Removes all dirty characters and from text following a regex pattern.    
- Dirty characters are things like !@#$%^&*()?>< etc..
- Useful for reducing dimension/variance of your data since fewer symbols will occur
- Useful for cleaning tweets 
- Matches slangs
- Language independent 
- You can use a regex pattern to specify which tokens will *not* be removed.  

I.e the pattern [a-z] matches all characters from a,b,c... to x,y,z. It will throw
```
pipe['normalizer'].setCleanupPatterns('[a-z]') 
```


# 1. Install Java and NLU

In [1]:

import os
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu  > /dev/null    

## 2. Load Model and normalize sample string

In [2]:
import nlu 


nlu.load('norm').predict('@CKL_IT says: that #normalizers are pretty useful to clean #structured_strings in #NLU like tweets')

,normalized,sentence
origin_index,,
0,"[CKLIT, says, that, normalizers, are, pretty, ...",@CKL_IT says: that #normalizers are pretty use...


## 2. Configure the normalizer with custom parameters
Use the pipe.print_info() to see all configurable parameters and infos about them for every NLU component in the pipeline pipeline.     
Even tough only 'norm' is loaded, many NLU component dependencies are automatically loaded into the pipeline and also configurable. 


By default the normalizer will set all tokens to lower case.     
Lets change that

In [3]:
pipe = nlu.load('norm')
pipe.predict('LOWERCASE BY DEFAULT')

,normalized,sentence
origin_index,,
0,"[LOWERCASE, BY, DEFAULT]",LOWERCASE BY DEFAULT


### 2.1 Print all parameters for all NLU components in the pipeline 


In [4]:
pipe.print_info()


The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['normalizer'] has settable params:
pipe['normalizer'].setCleanupPatterns(['[^\\pL+]'])  | Info: normalization regex patterns which match will be removed from token | Currently set to : ['[^\\pL+]']
pipe['normalizer'].setLowercase(False)               | Info: whether to convert strings to lowercase | Currently set to : False
pipe['normalizer'].setSlangMatchCase(False)          | Info: whether or not to be case sensitive to match slangs. Defaults to false. | Currently set to : False
>>> pipe['default_tokenizer'] has settable params:
pipe['default_tokenizer'].setTargetPattern('\S+')    | Info: pattern to grab from text as token candidates. Defaults \S+ | Currently set to : \S+
pipe['default_tokenizer'].setContextChars(['.', ',', ';', ':', '!', '?', '*', '-', '(', ')', '"', "'"])  | Info: character list used to separate from token boundaries | Currently set to : ['.', ',', ';', ':',

### 2.2 Configure the Normalizer not to lowercase text 

In [5]:
pipe['normalizer'].setLowercase(True)      
pipe.predict('LOWERCASE BY DEFAULT')

,normalized,sentence
origin_index,,
0,"[lowercase, by, default]",LOWERCASE BY DEFAULT


### 2.3Configure normalizer to remove strings based on regex pattern.
Lets remove all occurences of the lowercase letters x to z with the pattern [x-z]. 

In [6]:
# Configure the Normalizer 
pipe['normalizer'].setCleanupPatterns(['[x-z]']) 
pipe.predict('From the x to the y to the z')

,normalized,sentence
origin_index,,
0,"[from, the, to, the, to, the]",From the x to the y to the z


#### NOTE: The regex pattern is applied **BEFORE** lowercasing.    
This is why the X,Y,Z tokens are kept i nthe following example


In [7]:
# Configure the Normalizer 
pipe['normalizer'].setCleanupPatterns(['[x-z]']) 
pipe.predict('From the X to the Y to the Z')

,normalized,sentence
origin_index,,
0,"[from, the, x, to, the, y, to, the, z]",From the X to the Y to the Z


# 3. Get one row per normalized token by setting outputlevel to token.    
This lets us compare what the original token was and what it was normalized to. 

In [8]:
pipe.predict('From the X to the Y to the Z', output_level='token')

,normalized,token
origin_index,,
0,from,From
0,the,the
0,x,X
0,to,to
0,the,the
0,y,Y
0,to,to
0,the,the
0,z,Z
